In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from multirtd.dubins_model import dubins_traj

### Monte carlo

In [ ]:
# Initial state (no uncertainty)
x0 = np.array([0, 0, 0])  # [x, y, theta]

# Monte carlo over 1000 samples
dt = 1.0
N_MC = 1000
K = 100  # Number of timesteps
sigma = 0.03  # rotational velocity sigma

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity

x_samples = []
for i in range(N_MC):
    u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
    x = dubins_traj(x0, u, dt)
    x_samples.append(x)

# Plot the trajectory
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
for x in x_samples:
    ax1.plot(x[:,0], x[:,1], 'b', alpha=0.1)
    ax2.scatter(x[-1,0], x[-1,1])
plt.axis('equal')
plt.show()

In [ ]:
dt = 0.01
K = 100
sigma = 0.3

# Linearized dynamics
def linearize_dynamics(x, u, dt):
    A = np.eye(3)
    A[0, 2] = -u[0] * dt * np.sin(x[2])
    A[1, 2] = u[0] * dt * np.cos(x[2])
    B = np.zeros((3, 2))
    B[0, 0] = dt * np.cos(x[2])
    B[1, 0] = dt * np.sin(x[2])
    B[2, 1] = dt
    return A, B

u = np.zeros((K, 2))
u[:, 0] = 1  # constant velocity
#u[:, 1] = np.random.normal(0, sigma, K)  # random rotational velocity
u[:, 1] = 0.1
x = dubins_traj(x0, u, dt)

# Compare linearized dynamics with actual dynamics
x_lin = np.zeros((K, 3))
x_lin[0] = x0
for i in range(K-1):
    A, B = linearize_dynamics(x[i], u[i], dt)
    x_lin[i+1] = A @ x_lin[i] + B @ u[i]

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(10, 5))
ax1.axis('equal')
ax1.plot(x[:,0], x[:,1], 'b', label='Actual')
ax1.plot(x_lin[:,0], x_lin[:,1], 'r', label='Linearized')

### Closed loop simulation

In [ ]:
DT = 0.1  # Time step
N = 15  # Number of timesteps

# Nominal trajectory
x0 = np.array([0, 0, 0])
U = np.array([[1, 0.1]] * N)
traj = dubins_traj(x0, U, DT)
# Plot trajectory
plt.plot(traj[:, 0], traj[:, 1])
plt.axis('equal')
plt.show()

In [ ]:
def measurement(x, Sigma):
    """Measurement function
    
    (x,y,theta) -> (x,y) + noise
    
    """
    return x[:2] + np.random.multivariate_normal(np.zeros(2), Sigma)

In [ ]:
# Kalman filter
C = np.array([[1, 0, 0], [0, 1, 0]])  # Measurement matrix